## Create simple solar system model

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from collections import namedtuple

## Define planet class
Will allow me to collect stuff together for a planet

In [ ]:
class planet():
    "A planet in our solar system"
    def __init__(self,semimajor,eccentricity):
        self.x    =np.zeros(2)  #x and y position
        self.v    =np.zeros(2)  #x and y velocity
        self.a_g  =np.zeros(2)  #x and y acceleration
        self.t    = 0.0         #current time
        self.dt   = 0.0         #current time step
        self.a    = semimajor   #semimajor axis of the orbit
        self.e    = eccentricity #eccentricity of the orbit
        self.istep = 0   #current integer timestep
        self.name = ""   #name for the planet

## Define dictionary with some constants

In [ ]:
solar_system = { "M_sun":1.0, "G":39.4784176043574320}

## Define some functions for setting circular velocity and acceleration

In [ ]:
def SolarCircularVelocity(p):
    
    G = solar_system["G"]
    M = solar_system["M_sun"]
    r = ( p.x[0]**2 + p.x[1]**2 )**0.5
    
    #return circular velocity 
    return (G*M/r)**0.5

## Write fxn to compute the gravitational acceleration on each planet from the Sun

In [ ]:
def SolarGraviationalAcceleration(p):
    
    G = solar_system["G"]
    M = solar_system["M_sun"]
    r = ( p.x[0]**2 + p.x[1]**2 )**0.5
    
    #acceleration in AU/yr/yr 
    a_grav = -1.0*G*M/r**2
    
    #find the angle at this position
    if (p.x[0]==0.0):
        if (p.x[1]>0.0):
            theta = 0.5*np.pi
            
        else:
            theta = 1.5*np.pi
    else:
        theta = np.arctan2(p.x[1],p.x[0])
        
    #set the x and y components of velocity 
    #p.a_g[0] = a_grav * np.cos(theta)
    #p.a_g[1] = a_grav * np.sin(theta)
    return a_grav*np.cos(theta), a_grav*np.sin(theta)

## compute the timestep

In [ ]:
def calc_dt(p):
    #integration tolerance
    ETA_TIME_STEP = 0.0004
    
    #compute time step
    eta = ETA_TIME_STEP
    v = (p.v[0]**2 + p.v[1]**2)**0.5
    a = (p.a_g[0]**2 + p.a_g[1]**2)**0.5
    dt = eta * np.fmin(1./np.fabs(v), 1./np.fabs(a)**0.5)
    
    return dt

## Define intital conditions

In [ ]:
def SetPlanet(p, i):
    AU_in_km = 1.495979e+8 #an AU in km
    
    #circular velocity
    v_c = 0.0          #circular velocity in AU/yr
    v_e = 0.0          #velocity at perihelion in AU/yr
    
    #planet by planet initial conditions
    
    #Mercury 
    if(i==0):
        #semi-major axis in AU
        p.a = 57909227.0/AU_in_km
        
        #eccentricity
        p.e = 0.20563593
        
        #name
        p.name = "Mercury"
        
    #venus
    elif(i==1):
        #semimajor axis in AU
        p.a = 108209475.0/AU_in_km
        
        #eccentricity
        p.e = 0.00677672
        
        #name
        p.name = "Venus"
        
    #earth
    elif(i==2):
        #semimajor axis in AU
        p.a = 1.0
        
        p.e = 0.01671123
        
        p.name = "Earth"
        
    #set remaining properties
    p.t = 0.0
    p.x[0] = p.a*(1.0-p.e)
    p.x[1]= 0.0
    
    #get equiv circular velocity
    v_c = SolarCircularVelocity(p)
    
    #velocity at perihelion 
    v_e = v_c*(1 + p.e)**0.5
    
    #set velocity
    p.v[0] = 0.0     #no x velocity at perihelion
    p.v[1] = v_e     #y velocity at perihelion (counter clockwise)
    
    #caluclate gravitational acceleration from sun
    p.a_g = SolarGravitationalAcceleration(p)
    
    #set timestep
    p.dt = calc_dt(p)

## write leapfrog integrator